<a href="https://colab.research.google.com/github/priyanshgupta1998/Machine_learning/blob/master/Kaggle/NLP-chatbot_Processing_for_kaggle_Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Question-Answer Dataset
Can you use NLP to answer these questions?

In [2]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
!kaggle datasets download -d rtatman/questionanswer-dataset

  0% 0.00/3.55M [00:00<?, ?B/s]
100% 3.55M/3.55M [00:00<00:00, 116MB/s]


In [0]:
!unzip questionanswer-dataset.zip

`Activity: Write a chatbot that, given a question in English, finds a similar question in the Q&A corpus available at Kaggle (https://www.kaggle.com/rtatman/questionanswer-dataset#S08_question_answer_pairs.txt) and the answer.`

`The "question_answer_pairs.txt" files contain the questions and answers. The columns in this file are as follows:`

`ArticleTitle is the name of the Wikipedia article from which the questions and answers initially arose. Question is the question. Answer is the answer. DifficultyFromQuestioner is the difficulty rating prescribed for the question given to the question's author. AnnswererDifficulty is a difficulty rating assigned by the individual who assessed and answered the question, which may differ from the difficulty in field 4. ArticleFile is the file name with the relevant article Questions that were judged as bad were discarded from this dataset.`

`There are usually several lines with the same question that appear if these questions have been answered by multiple individuals.`

In [34]:
import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize

import codecs
#doc = codecs.open('document','rU','UTF-16') #open for reading with "universal" type set
#df = pandas.read_csv(doc, sep='\t')
#https://stackoverflow.com/questions/27896214/reading-tab-delimited-file-with-pandas-works-on-windows-but-not-on-mac
doc = codecs.open('S08_question_answer_pairs.txt','rU','UTF-8')
S08 = pd.read_csv(doc, sep="\t", header=None, names=["ArticleTitle","Question", "Answer","DifficultyFromQuestioner","DifficultyFromAnswerer","ArticleFile"])
S08.drop([0] , axis=0, inplace = True)
print(S08.shape)

doc = codecs.open('S09_question_answer_pairs.txt','rU','UTF-8')
S09 = pd.read_csv(doc, sep="\t", header=None, names=["ArticleTitle","Question", "Answer","DifficultyFromQuestioner","DifficultyFromAnswerer","ArticleFile"])
S09.drop([0] , axis=0, inplace = True)
print(S09.shape)

doc = codecs.open('S10_question_answer_pairs.txt','rU','ISO-8859-1')
S10 = pd.read_csv(doc, sep="\t", header=None, names=["ArticleTitle","Question", "Answer","DifficultyFromQuestioner","DifficultyFromAnswerer","ArticleFile"])
S10.drop([0] , axis=0, inplace = True)
print(S10.shape )


data  = pd.concat([S08, S09, S10] , ignore_index=True)

(1715, 6)
(825, 6)
(1458, 6)


In [35]:
S08_raw = data.copy()
print(S08_raw.shape)
S08_raw.head()

(3998, 6)


,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,S08_set3_a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,S08_set3_a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,S08_set3_a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,S08_set3_a4
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,S08_set3_a4


In [36]:
S08_cleaned = S08_raw[pd.notnull(S08_raw['Question'])]
print(S08_cleaned.shape)  # 37 data had null values

(3961, 6)


In [38]:
S08_cleaned = S08_cleaned.drop_duplicates(subset=['Question'], keep='first', inplace=False)
print(S08_cleaned.shape) # find the unique data 

(2456, 6)


In [39]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk import tokenize
import string
from string import punctuation

stopwords_list = nltk.corpus.stopwords.words('english')
tokens = tokenize.WhitespaceTokenizer()
question_cleaned = list()
for question in S08_cleaned['Question']:  
    new_question = list()
    word_text = tokens.tokenize(question)
    for word in word_text:
        if word not in stopwords_list:
            #new_question.append(remove_punctuation(word.lower()))
            new_question.append(''.join([c for c in word.lower() if c not in punctuation]))
    question_cleaned.append(' '.join(new_question))
S08_cleaned['Question_Cleaned'] = question_cleaned  

In [43]:
#remove first row (single row) from the daatset 
# S08_cleaned.drop([0] , axis=0, inplace = True)
print(S08_cleaned.shape)
S08_cleaned.head()

(2456, 7)


,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile,Question_Cleaned
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,S08_set3_a4,was abraham lincoln sixteenth president united...
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,S08_set3_a4,did lincoln sign national banking act 1863
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,S08_set3_a4,did mother die pneumonia
6,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months,medium,easy,S08_set3_a4,how many long lincolns formal education
8,Abraham_Lincoln,When did Lincoln begin his political career?,1832,medium,easy,S08_set3_a4,when lincoln begin political career


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(S08_cleaned['Question_Cleaned'].values)
print(tfidf_matrix.shape)

(2456, 4515)


In [57]:
print(len(tfidf_matrix.data))
print(tfidf_matrix[:5].data)
print(' ')
for i in range(5):
  print('Question_{} -->'.format(i) , tfidf_matrix[i].data)

14850
[0.36077749 0.36410492 0.32680186 0.49932743 0.35175516 0.45666948
 0.22104563 0.45725985 0.38665718 0.45725985 0.32749177 0.41819577
 0.21546002 0.32212032 0.6546307  0.46491318 0.51006502 0.3084608
 0.45874557 0.50601272 0.50601272 0.37092409 0.27638978 0.2519499
 0.53520568 0.46911708 0.48521158 0.2968246  0.41224861]
 
Question_0 --> [0.36077749 0.36410492 0.32680186 0.49932743 0.35175516 0.45666948
 0.22104563]
Question_1 --> [0.45725985 0.38665718 0.45725985 0.32749177 0.41819577 0.21546002
 0.32212032]
Question_2 --> [0.6546307  0.46491318 0.51006502 0.3084608 ]
Question_3 --> [0.45874557 0.50601272 0.50601272 0.37092409 0.27638978 0.2519499 ]
Question_4 --> [0.53520568 0.46911708 0.48521158 0.2968246  0.41224861]


In [0]:
from sklearn.metrics.pairwise import cosine_similarity

question = ""
i = 0
while question!="\0":
    question = input("What is your question?: ")
    if question != "\0":
        query_vect = tfidf_vectorizer.transform([question])
        l = cosine_similarity(query_vect, tfidf_matrix)
        print(type(l))
        while l.max() < 0.5:
            question = input("Could you be more specific please? What is your question?: ")
            query_vect = tfidf_vectorizer.transform([question])
            if question != "\0":
                l = cosine_similarity(query_vect, tfidf_matrix)
            else:                
                break
            
        if question != "\0":
            i = np.where(l == l.max())
            #print(S08_cleaned.iloc[(i[1][0])][['Question','Answer']])
            print(S08_cleaned.iloc[(i[1][0])][['Answer']])
        
            
print("volte sempre, agradeçemos a visita") 



---



---




---



---



---



---



---


#Solve above problem using another method

In [0]:
import pandas as pd
import numpy as np
import string
df_S08 = pd.read_csv('S08_question_answer_pairs.txt', sep="\t", header=0)
df_S09 = pd.read_csv('S09_question_answer_pairs.txt', sep="\t", header=0)
df_S10 = pd.read_csv('S10_question_answer_pairs.txt', sep="\t", header=0, encoding = "ISO-8859-1")

#Data Preprocessing

In [0]:
df_list = ['df_S08','df_S09','df_S10']
questions = []
answer = []
for d in df_list:
    d = eval(d)
    d['Question'] = d['ArticleTitle'] + " " + d['Question'] # concatena nome artigo em pergunta para melhorar referencia
    d = d[['Question','Answer']] #remove colunas indesejadas
    d = d.dropna() # Remove NaN
    d['Answer'] = d['Answer'].str.lower() # deixa tudo minusculo
    d['Answer'] = d['Answer'].str.replace('[{}]'.format(string.punctuation), '') # remove pontuacao
    d['Answer'] = d['Answer'].str.replace(r'\\t', '') #remove qualquer "\t" que existir em respostas
    d['Question'] = d['Question'].str.replace(r'_', ' ') #remove qualquer "_" que existir em perguntas
    d = d.sort_values('Question').drop_duplicates(subset=['Question', 'Answer'], keep='last') #remove dados duplicados
    questions = np.append(questions, d['Question'].values) #cria np array de questoes
    answer = np.append(answer, d['Answer'].values) #cria np array de respostas

#Finding the most similar answer

In [0]:
perguntas = tuple(questions)
pergunta = ["Does Arabic language have many words borrowed by European languages?"]

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(perguntas)

from sklearn.metrics.pairwise import cosine_similarity
query_vect = tfidf_vectorizer.transform(pergunta)
similariedade = cosine_similarity(query_vect, tfidf_matrix) # calcula e cria um vetor com os valores de similariedade

indice = np.unravel_index(np.argmax(similariedade, axis=None), similariedade.shape) #encontra o indice do valor com maior similariedade

print("RESPOSTA: {}".format(answer[indice[1]]))
print("PERGUNTA ORIGINAL: {}".format(questions[indice[1]]))

#Let's Check the Question and find most relevant answer

In [0]:
display(df_S09.Question[100])
display(df_S09.Answer[100])




---


---



---



---



---




---



---



---



---


#Solve above problem using another method

In [82]:
dfS08_question= pd.read_csv('S08_question_answer_pairs.txt',  sep="\t")
#Deleting columns not containing questions
dfS08_question.drop(labels=['ArticleTitle', 'DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile', 'Answer'], axis=1, inplace=True)
dfS08_question.head()

,Question
0,Was Abraham Lincoln the sixteenth President of...
1,Was Abraham Lincoln the sixteenth President of...
2,Did Lincoln sign the National Banking Act of 1...
3,Did Lincoln sign the National Banking Act of 1...
4,Did his mother die of pneumonia?


#Converting Data Frame to txt and then import as text array into Numpy

In [88]:
import nltk
import os
import numpy as np
from nltk.tokenize import sent_tokenize

dfS08_question.to_csv("csvfile.txt", sep=";", index=False, header=False)
print(os.listdir("."))

['.config', 'csvfile.txt', 'kaggle.json', 'S08_question_answer_pairs.txt', 'text_data', 'S09_question_answer_pairs.txt', 'text_data_toc.csv', 'S10_question_answer_pairs.txt', 'questionanswer-dataset.zip', 'sample_data']


In [90]:
rawText = np.genfromtxt("/content/csvfile.txt", dtype='str', delimiter=';', usecols=np.arange(0,1))
print(rawText.shape)
rawText[:5]

(1715,)


array(['Was Abraham Lincoln the sixteenth President of the United States?',
       'Was Abraham Lincoln the sixteenth President of the United States?',
       'Did Lincoln sign the National Banking Act of 1863?',
       'Did Lincoln sign the National Banking Act of 1863?',
       'Did his mother die of pneumonia?'], dtype='<U252')

# we will complete it later

#Other 
NLTK LIBRARY

`Now let's define a function for scikit-learn tokenization.`

In [0]:
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import wordnet

stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

#And this function will be called by the TfidfVectorizer object.
def my_tokenizer(doc):
    
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]

    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas
    

In [67]:
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download('gutenberg')
nltk.download('punkt')
hamlet_raw = nltk.corpus.gutenberg.raw('shakespeare-hamlet.txt')

sents = sent_tokenize(hamlet_raw)

hamlet_np = np.array(sents)

print(hamlet_np.shape)

from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)
tfs = tfidf_vectorizer.fit_transform(hamlet_np)
print(tfs.shape)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
(2355,)
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
(2355, 4305)


In [68]:
print([k for k in tfidf_vectorizer.vocabulary_.keys()][:20])

['tragedie', 'hamlet', 'william', 'shakespeare', '1599', 'actus', 'primus', 'scoena', 'prima', 'enter', 'barnardo', 'francisco', 'two', 'centinels', "'s", 'fran', 'nay', 'answer', 'stand', 'vnfold']


In [70]:
print(tfs[:50,:50])

  (0, 17)	0.4098405429500896
  (4, 5)	1.0
  (12, 6)	0.25569565102708125
  (13, 6)	0.24183188573710981
  (22, 5)	0.5865767987601467
  (27, 0)	0.22677320532838496
  (29, 5)	0.28458175550618814
  (37, 0)	0.23602778459784524
  (37, 5)	0.26402019659267234
  (39, 6)	0.13875455587670746
  (40, 13)	0.35297267044370917
  (41, 25)	0.31475626155215114
  (43, 5)	0.14416024946136125
  (46, 5)	0.32299630350190056


`Optionally, we can get the tf-idf attributes from n-grams by combining the CountVectorizer and TfidfTransformer classes. In our example, let's just use trigrams:`

In [71]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(ngram_range=(3,3)) #trigrams

n_gram_counts = count_vect.fit_transform(hamlet_np)

tfidf_transformer = TfidfTransformer()

tfs_ngrams = tfidf_transformer.fit_transform(n_gram_counts)

print(tfs_ngrams.shape)

(2355, 23483)


#Dimensionality reduction
`The transformation of the corpus into attributes containing the TF-IDF frequencies will generally result in a very sparse ndarray, ie with many dimensions. However, in addition to making algorithm training longer and more costly (computationally speaking), many of these dimensions are probably unrepresentative or even noisy during training. To solve this problem, we can apply a simple dimensionality reduction technique called Singular Value Decomposition (SVD).`

`This technique will transform the vectors of the original matrix, rotating and scaling them, resulting in new representations. The dimensionality reduction is made by keeping only the most representative k dimensions we choose. Another advantage of this technique is that the original dimensions are somewhat "combined," resulting in a new way of representing the combination of terms. In the context of PLN, this technique is known as Latent Semantic Analysis (LSA).`


In [72]:
from sklearn.decomposition import TruncatedSVD

svd_transformer = TruncatedSVD(n_components=1000)

svd_transformer.fit(tfs)

print(sorted(svd_transformer.explained_variance_ratio_)[::-1][:30])

[0.04319143964864231, 0.018619648885290326, 0.015991826821457302, 0.015984996712032823, 0.013744333318751852, 0.01177158023826238, 0.009650213667974963, 0.009516807424959752, 0.009505050204169866, 0.009460542064127395, 0.009317453675253258, 0.00927004256530776, 0.00771140689119979, 0.006983997622746521, 0.006654688885440767, 0.006419528576285735, 0.005885302030105491, 0.005372277900306946, 0.00515817119066351, 0.00493125795647099, 0.004867527112014457, 0.004789736065695093, 0.004462492864893638, 0.004339957865165943, 0.004296902243616561, 0.004160561728405304, 0.004039480646183629, 0.003913733369732403, 0.0038585289997894383, 0.0038298284368150155]


`Now let's keep the dimensions until the cumulative variance is greater than or equal to 0.50.`

In [73]:
cummulative_variance = 0.0
k = 0
for var in sorted(svd_transformer.explained_variance_ratio_)[::-1]:
    cummulative_variance += var
    if cummulative_variance >= 0.5:
        break
    else:
        k += 1
        
print(k)

143


`We transform again, but this time with the number of k components we obtained earlier.`

In [74]:
svd_transformer = TruncatedSVD(n_components=k)
svd_data = svd_transformer.fit_transform(tfs)
print(sorted(svd_transformer.explained_variance_ratio_)[::-1])

[0.04319143964849607, 0.01861964888757188, 0.015991826832433435, 0.01598499671092047, 0.013744333326091625, 0.01177158028827747, 0.009650213469195786, 0.009516807243291651, 0.009505050265448222, 0.009460542005755143, 0.00931745368020356, 0.009270042542963708, 0.007711406071217975, 0.006983997677641885, 0.006654689740897062, 0.006419529301423177, 0.005885298834675811, 0.005372276987945273, 0.0051581704075479955, 0.00493125947141876, 0.004867526342691276, 0.004789733790767578, 0.0044624929025246065, 0.0043399473697041885, 0.004296900685370986, 0.004160556529847263, 0.00403947342661859, 0.003913725702760502, 0.0038585073180558947, 0.0038298265648413986, 0.003713634254731037, 0.003663393169366966, 0.0035824742431858313, 0.0035617258103456313, 0.0035351640190848274, 0.0034284355602272405, 0.0033450756885336787, 0.003242946882513592, 0.0032341297071723, 0.0031762222377768588, 0.0031226652600258396, 0.0030422567387544735, 0.002931403643054755, 0.0029182733310599636, 0.00288770944523902, 0.002

In [75]:
print(svd_data.shape)

(2355, 143)


#Cosine Similarity

`Since two documents are represented as numeric vectors, it is possible to compare the similarity between documents by calculating the cosine of the angle between these documents - a measure of distance, not amplitude. To do this, simply solve the scalar product equation between the vectors to find the cosine.`    

![alt text](http://s0.wp.com/latex.php?latex=++%5Cdisplaystyle++%5Cvec%7Ba%7D+%5Ccdot+%5Cvec%7Bb%7D+%3D+%5C%7C%5Cvec%7Ba%7D%5C%7C%5C%7C%5Cvec%7Bb%7D%5C%7C%5Ccos%7B%5Ctheta%7D+%5C%5C+%5C%5C++%5Ccos%7B%5Ctheta%7D+%3D+%5Cfrac%7B%5Cvec%7Ba%7D+%5Ccdot+%5Cvec%7Bb%7D%7D%7B%5C%7C%5Cvec%7Ba%7D%5C%7C%5C%7C%5Cvec%7Bb%7D%5C%7C%7D++&bg=ffffff&fg=000000&s=0)   


>>> ![alt text](http://blog.christianperone.com/wp-content/uploads/2013/09/vector_space.png)

In [76]:
 documents = (
    "The sky is blue",
    "The sun is bright",
    "The sun in the sky is bright",
    "We can see the shining sun, the bright sun"
    )

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
print(tfidf_matrix.shape)

(4, 11)


In [78]:
from sklearn.metrics.pairwise import cosine_similarity

query_vect = tfidf_vectorizer.transform(["The sun is red in the sky"])

cosine_similarity(query_vect, tfidf_matrix)

array([[0.55245238, 0.59222761, 0.9467919 , 0.37986898]])